In [154]:
from functools import partial
import os
from sqlite3 import connect

import astroplan as ap
import astropy.units as u
from icecream import ic, install
import numpy as np

from astropaul.observing import ObservingSession
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr
import astropaul.targetlistcreator as tlc

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [242]:
session = ObservingSession(observer=ap.Observer(latitude=38.05246 * u.deg, longitude=-78.49567 * u.deg, timezone="US/Eastern"))
# session.add_full_day("2024-12-22")
session.add_day_range("2024-12-21", "2024-12-28")
html_dir = "html"

phase_event_defs = [
    # ph.PhaseEventDef("Mid Eclipse", partial(ph.calc_time_of_phase, phase=0.0)),
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

creator = tlc.TargetListCreator(name="Paul Light Curves", connection=connect("../../django/TargetDB/db.sqlite3"))
creator.steps = [
    tlc.add_targets,
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    tlc.add_lists,
    tlc.add_speckle,
    tlc.add_pepsi,
    # partial(tlc.filter_targets, criteria=lambda df: (df["Target Name"] == "TIC 367448265")),
    partial(tlc.filter_targets, criteria=lambda df: (df["Num Speckle"] > 0)),
    partial(tlc.filter_targets, criteria=lambda df: (df["Num PEPSI"] > 0)),
    partial(tlc.add_observability, observing_session=session, calc_max_altitude=True, calc_moon_distance=True),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Every Night"])),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Max Alt"] > 40)),
    tlc.add_ephemerides,
    partial(tlc.add_phase_events, observing_session=session, phase_event_defs=phase_event_defs),
]

tl = creator.calculate()
tl.to_html(f"{html_dir}/{tl.name}.html")
print(tl.summarize())
tl.target_list

Paul Light Curves
23 targets:
      23 QuadEB
Column Count (primary, secondary):
    Target: (1, 2)
    Coordinates: (6, 3)
    List : (0, 10)
    Count: (2, 0)
    Observable: (4, 21)
Associated tables:
     149 rows,  6 columns: Speckle
     103 rows,  9 columns: PEPSI
       7 rows,  2 columns: Lunar Phases
      78 rows,  6 columns: Ephem
     796 rows,  8 columns: Phase Events



,Target Name,Target Source,Target Type,ra,dec,pmra,pmdec,parallax,Vmag,Teff,...,Observable 2024-12-25,Observable 2024-12-25 Max Alt,Observable 2024-12-25 Min Moon Dist,Observable 2024-12-26,Observable 2024-12-26 Max Alt,Observable 2024-12-26 Min Moon Dist,Observable Any Night,Observable Every Night,Observable Max Alt,Observable Min Moon Dist
id,,,,,,,,,,,,,,,,,,,,,
4210,TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,223.425163,52.715848,2.17801,-3.594290,2.027610,12.497,5860.00,...,True,69.768732,70.130478,True,70.310329,73.965861,True,True,70.310329,64.350096
4215,TIC 367448265,Kostov 2022 arXiv:2202.05790,QuadEB,78.382438,35.653053,-5.85675,-3.432870,3.264150,7.900,9212.00,...,True,87.603020,135.327818,True,87.305665,146.995436,True,True,87.618723,76.874400
4219,TIC 25818450,Kostov 2022 arXiv:2202.05790,QuadEB,352.743444,53.069150,3.57360,-1.172130,1.065210,11.782,7172.00,...,True,74.840276,132.242466,True,74.828875,129.617004,True,True,74.841086,115.690840
4223,TIC 414026507,Kostov 2022 arXiv:2202.05790,QuadEB,336.837717,56.740362,-3.66449,-2.956470,0.242427,10.899,9187.00,...,True,70.416184,122.458720,True,70.192893,119.915075,True,True,71.104663,112.987007
4225,TIC 470710327,Kostov 2022 arXiv:2202.05790,QuadEB,357.329052,61.962787,0.60000,-2.500000,NaN,9.650,8986.00,...,True,65.936570,128.476900,True,65.945986,128.217172,True,True,65.945986,106.468827
4228,TIC 391620600,Kostov 2022 arXiv:2202.05790,QuadEB,71.640389,44.753662,-7.75529,2.230940,2.072980,12.772,5949.71,...,True,83.246724,137.023103,True,83.225889,147.478404,True,True,83.252033,81.300942
4229,TIC 89278612,Kostov 2022 arXiv:2202.05790,QuadEB,301.219498,32.643051,-0.48521,-1.506920,0.663315,11.235,8740.00,...,True,54.224346,97.810160,True,53.324510,91.000295,True,True,58.649485,91.000295
4231,TIC 278352276,Kostov 2022 arXiv:2202.05790,QuadEB,307.503640,48.607056,1.85387,6.576640,1.362650,10.387,7156.00,...,True,61.047221,105.689521,True,60.295035,101.456081,True,True,64.705873,101.456081
4232,TIC 123098844,Kostov 2022 arXiv:2202.05790,QuadEB,279.572843,44.698600,3.48250,-9.104270,1.309250,11.136,6761.00,...,True,41.733511,86.364314,True,40.951647,82.946344,True,True,45.602816,82.946344


In [248]:
illumination_categories = [
    ((0.0, 0.4), "Dark"),
    ((0.4, 0.7), "Gray"),
    ((0.7, 1.0), "Bright"),
]

distance_categories = {
    "Dark": [
        ((0, 180), 1),
    ],
    "Gray": [
        ((0, 5), 0.1),
        ((5, 15), 0.75),
        ((15, 180), 1),
    ],
    "Bright": [
        ((0, 15), 0.25),
        ((15, 30), 0.75),
        ((30, 180), 1),
    ],
}

altitude_categories = [
    ((-90, 35), 0),
    ((35, 45), 0.85),
    ((45, 60), 0.95),
    ((60, 90), 1),
]

# phase_defs = [
#     ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
#     ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
#     ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
#     ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
#     ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
#     ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
#     ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
#     ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
#     ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
# ]

phase_scores = {
    "B|B": 0.5,
    "B|R": 0.8,
    "B|T": 0.4,
    "R|R": 0.9,
    "R|T": 1,
    "T|T": 0.1,
}

pl = pr.PriorityList(tl, session, interval=60 * u.min)
# pr.calculate_moon_priority(pl, illumination_categories=illumination_categories, dist_categories=distance_categories)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories, min_nonzero_time=6 * u.hour)
# pr.calculate_phase_priority(pl, phase_defs=phase_defs, phase_categories=phase_scores)
pr.calculate_eclipse_priority(pl, no_eclipse_score=0.2, min_altitude=35 * u.deg)
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.1)
pl.categorize_priorities(
    bins=[0.00, 0.10, 0.5, 0.80, 1.00], labels=["", "*", "* *", "* * *"]
)
html.render_observing_pages(tl, pl, {}, "html")
# pl.numerical_priorities[2]
# pl.target_tables["TIC 367448265"][0]
# pl.numerical_priorities[0]

In [245]:
from playwright.sync_api import sync_playwright


def html_to_pdf_only_body(input_html_path, output_pdf_path):
    with sync_playwright() as p:
        browser = p.chromium.launch()
        page = browser.new_page()

        # Load the HTML file
        page.goto(f"file://{input_html_path}")

        # Remove everything except the <body>
        page.evaluate(
            """() => {
            document.body.innerHTML = document.body.innerHTML;
        }"""
        )

        # Generate the PDF
        page.pdf(path=output_pdf_path, format="Letter")
        print(f"PDF saved to {output_pdf_path}")

        # Close the browser
        browser.close()


# Call the function
html_to_pdf_only_body("C:/Users/User/Dropbox/Astro/Other/2024-10-30 Paul Light Curves/html/Categorical Priorities 2024-12-20.html", "foo.pdf")

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [210]:
html.render_observing_pages(tl, pl, {}, "html")
# pl.numerical_priorities[2]
pl.target_tables["TIC 367448265 "][0]
# pl.numerical_priorities[0]

,Altitude Value,Altitude Priority,System A Eclipse,System A Problems,System B Eclipse,System B Problems,Eclipse Priority,Overall Priority
2024-12-12 22:00:00,11.052854,0.00,,,,,0.2,0.00
2024-12-12 22:30:00,15.898701,0.00,,,,,0.2,0.00
2024-12-12 23:00:00,20.954907,0.00,,,,,0.2,0.00
2024-12-12 23:30:00,26.190787,0.00,,,,,0.2,0.00
2024-12-13 00:00:00,31.579585,0.00,,,,,0.2,0.00
2024-12-13 00:30:00,37.097940,0.85,,,,,0.2,0.17
2024-12-13 01:00:00,42.725343,0.85,,,,,0.2,0.17
2024-12-13 01:30:00,48.443586,0.95,,,,,0.2,0.19
2024-12-13 02:00:00,54.236171,0.95,,,,,0.2,0.19
2024-12-13 02:30:00,60.087573,1.00,,,,,0.2,0.20


In [38]:
import jinja2

all_targets_creator = tlc.TargetListCreator(name="Paul", connection=connect("../../django/TargetDB/db.sqlite3"))
all_targets_creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Target Type"] == "White Dwarf Binary"),
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
]

all_targets = all_targets_creator.calculate()
all_targets.target_list

loader = jinja2.FileSystemLoader(".")
env = jinja2.Environment(loader=loader)
template = env.get_template("NINA Target template.jinja2")
for _, target in all_targets.target_list.iterrows():
    target_name = target["Target Name"]
    vars = {}
    vars["target_name"] = target_name
    vars["RA_hour"] = target["RA"][0:3]
    vars["RA_min"] = target["RA"][3:6]
    vars["RA_sec"] = target["RA"][6:]
    vars["DEC_sign"] = target["Dec"][0:1]
    vars["DEC_deg"] = target["Dec"][1:3]
    vars["DEC_min"] = target["Dec"][4:6]
    vars["DEC_sec"] = target["Dec"][7:]
    with open(f"{target_name}.json", "w") as f:
        f.write(template.render(vars))
    print(target_name)


TIC 270360534
TIC 219469945
TIC 20212631
TIC 150055835
TIC 161043618
TIC 58539786
TIC 441794509
TIC 219006972
TIC 304713857
TIC 367448265
TIC 160514618
TIC 240256832
TIC 120911334
TIC 25818450
TIC 346000664
TIC 430745185
TIC 371583423
TIC 414026507
TIC 121511673
TIC 470710327
TIC 269811101
TIC 282005870
TIC 391620600
TIC 89278612
TIC 63459761
TIC 278352276
TIC 123098844
TIC 273919067
TIC 358422952
TIC 278465736
TIC 322727163
TIC 375325607
TIC 344541836
TIC 327885074
TIC 427092089
TIC 265274458
TIC 417752064
TIC 387288959
TIC 264402353
TIC 260056937
TIC 266395331
TIC 266771301
TIC 265684107
TIC 274481742
TIC 470397849
TIC 9493888
TIC 79140936
TIC 78568780
TIC 80914862
TIC 60584909
TIC 13021681
TIC 146435300
TIC 93373156
TIC 24700485
TIC 405114468
TIC 409682934
TIC 125583594
TIC 73296637
TIC 266657256
TIC 443862276
TIC 177810207
TIC 32945664
TIC 36439758
TIC 53158069
TIC 317863971
TIC 261940513
TIC 262755078
TIC 43472931
TIC 200094011
TIC 244279814
TIC 38542448
TIC 454140642
TIC 45995991